In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import pathlib as path
import matplotlib.pyplot as plt
import math

import geopandas as gpd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import contextily as ctx

from src import parameters as params
from src import utils

## Read data

Read in the data provided. 
Update the parameters file to specify the location of the data files locally.

In [ ]:
df = pd.read_csv(params.DATA_DIR / params.data_file_name)

The dataset is GP level, with one record for each GP:

In [ ]:
len(df) == len(df.gp_code.drop_duplicates())

## Derive performance metrics

From the fields in the dataset, we derive a variety of performance metrics.

For each metric, we append a `(key, value)` pair to the dictionary `params.column_display_names` specifying the formatting of the column name for use in data visualisations and tables.

### Efficiency

Here we calculate the number of appointments each GP offered in the month of September 2024 per GP (Full Time Equivalent) employed by the practice. 

In some cases the number of GPs < 1 (due to some GPs working part-time or a data quality issue). This causes the number of appointments per gp to be greater than the number of appointments and causes outliers.

To handle this, we replace any outliers (appts_per_gp > Q3 + 1.5*IQR) with the median.

In [ ]:
df['patients_per_gp'] = df['numberofpatients'] / (np.ceil(df['qualified_gp']) + np.ceil(df['training_gp']))
params.column_display_names['patients_per_gp'] = 'Patients per GP'


df['appts_per_gp'] = (df['AttendanceOutcome_Attended']) / (np.ceil(df['qualified_gp']) + np.ceil(df['training_gp']))
#replace outliers with median
df['appts_per_gp'] = df['appts_per_gp'].apply(lambda x: df['appts_per_gp'].median() if x > df['appts_per_gp'].quantile(0.75) + 1.5*(df['appts_per_gp'].quantile(0.75) - df['appts_per_gp'].quantile(0.25)) else x)
params.column_display_names['appts_per_gp'] = 'Appointments per GP'

efficiency_metrics = ['appts_per_gp']

In [ ]:
fig, ax = plt.subplots(1, 4)
sns.boxplot(df.patients_per_gp.rename(params.column_display_names['patients_per_gp']), ax=ax[3])
sns.boxplot(df.appts_per_gp.rename(params.column_display_names['appts_per_gp']), ax=ax[0])
sns.boxplot(df.training_gp.rename(params.column_display_names['training_gp']), ax=ax[1])
sns.boxplot(df.qualified_gp.rename(params.column_display_names['qualified_gp']), ax=ax[2])

plt.tight_layout()

### Waiting times (Same day appointments)

Here, we obtain the percentage of attended appointments booked on the same day in September 2024.

In [ ]:
df['same_day_appointment_percentage'] = df['BookingtoApptGap_SameDay'] / (df['AttendanceOutcome_Attended'] + df['AttendanceOutcome_Unknown'])
params.column_display_names['same_day_appointment_percentage'] = 'Same-day Appointments (%)'

waiting_times_metrics = ['same_day_appointment_percentage']

### Digital Access

Percentage of appointments that were delivered over the phone or video call in September 2024.

In [ ]:
df['digital_access_percentage'] = (df['ApptModality_Telephone'] + df['ApptModality_VideoConferenceOnline']) / (df['AttendanceOutcome_Attended'] + df['AttendanceOutcome_Unknown'])
params.column_display_names['digital_access_percentage'] = 'Digital Access (%)'

digital_access_metrics = ['digital_access_percentage']

### Attendance rate

Percentage of appointments in September that were attended (not missed).

In [ ]:
df['attendance_rate'] = df['AttendanceOutcome_Attended'] / (df['AttendanceOutcome_Attended'] + df['AttendanceOutcome_DNA'])
params.column_display_names['attendance_rate'] = 'Attendance Rate (%)'

attendance_metrics = ['attendance_rate']

### Quality and Outcomes Framework (QOF)

Processing of QoF variables to be in the range $(0, 1)$ rather than percentages.

In [ ]:
df['qof_total'] = df.Total_QoF / 100
params.column_display_names['qof_total'] = 'QoF Total (%)'

df['qof_hypertension'] = df.Hypertension / 100
params.column_display_names['qof_hypertension'] = 'QoF Hypertension (%)'

df['qof_breast_cancer_screening'] = df.BreastScreeningCancer / 100
params.column_display_names['qof_breast_cancer_screening'] = 'QoF Breast Cancer Screening (%)'

df['qof_child_vaccination'] = df.ChildVaccination / 100
params.column_display_names['qof_child_vaccination'] = 'QoF Child Vaccination (%)'

qof_metrics = ['qof_total', 'qof_hypertension', 'qof_child_vaccination', 'EmergencyPresentationsCancer', 'AntibioticPrescribing']

### Patient experience / satisfaction (GP Survey)

No processing required for the GP survey columns

In [ ]:
gp_survey_metrics = ['overallexp', 'lastgpapptneeds', 'lastgpapptwait', 'localgpservicesreception', 'gpcontactoverall']

### CQC Ratings

For the ordinal CQC variables, a numerical encoding is applied.

In [ ]:
cqc_rating_encoding = {'Outstanding': 4, 'Good': 3, 'Requires improvement': 2, 'Inadequate': 1}

cqc_rating_columns = ['overall', 'responsive', 'wellled', 'effective', 'caring', 'safe']

cqc_metrics = []
for column in cqc_rating_columns:
    cqc_metrics.append(f'{column}_coded')
    df[f'{column}_coded'] = df[column].apply(lambda x : cqc_rating_encoding[x] if x in cqc_rating_encoding.keys() else np.nan)
    params.column_display_names[f'{column}_coded'] = params.column_display_names[column]

In [ ]:
performance_metrics = efficiency_metrics + waiting_times_metrics + digital_access_metrics + attendance_metrics + qof_metrics + gp_survey_metrics + cqc_metrics

## Filter to just North Central London (NCL) ICB

Filtering to just looking at GPs in NCL leaves us with 175 records, as specified in the Task Description.

In [ ]:
df_ncl = df[df.icb_code == params.ncl_icb]

len(df_ncl) == 175

## Multi-criteria Decision Analysis (MCDA)


The idea is to derive a composite score for each GP in NCL based on the performance metrics listed above. 
We will then use this composite score to rank the GPs where a lower score indicates lower performance.
More information and background on this technique can be found at:

[An Introductory Guide to Multi-Criteria Decision Analysis (MCDA) - Government Analysis Function](https://analysisfunction.civilservice.gov.uk/policy-store/an-introductory-guide-to-mcda/).

Let us define:

$$
C := \sum f_i(x_i) w_i
$$

where $f_i$ is some normalisation function for metric $x_i$ and $w_i$ is a weight assigned to the metric.

The weights $w_i$ are an indication of how important the metric is. Engagement with stakeholders can be used to determine the weights used. 
For example, the team at NCL ICB might be particularly interested in patient clinical outcomes, in which case we would assign a higher weight the Quality and Outcomes Framework (QoF) metrics.
Similarly, if a stakeholder is most interested in efficiency, we could assign more weight to the appointments per GP metric we defined above. 

### Missing Data

First, we must deal with missing values for these performance metrics. We will impute the NCL median when a performance metric is unavailable.

In [ ]:
df_ncl[params.performance_metrics.keys()].isna().sum()

In [ ]:
df_ncl_median_imputed = df_ncl.copy(deep=True)
df_ncl_median_imputed[list(params.performance_metrics.keys())] = df_ncl[params.performance_metrics.keys()].fillna(df_ncl[params.performance_metrics.keys()].median())

### Scaling / Normalisation of Metrics

We normalise the performance metrics to all have the same scale. This is so that when we aggregate them to obtain a composite score, they all carry the same weight. We can then use our custom defined weights to specify metric importance.

**Min-max normalization** is a scaling technique used to rescale a dataset so that all values fall within a specific range, typically between 0 and 1. It is particularly useful when you need to compare variables that are measured on different scales (e.g., percentages vs. counts).

**Formula for Min-Max Normalization**
For a given field $ y $ and GP $j$ in the dataset, the normalised value of $y$ for GP $j$ is given by:
$$
y_j' = \frac{y_j - \min(y)}{\max(y) - \min(y)}
$$

- $ y_i $: The original value.
- $\min(y) $: The minimum value of the field across all GPs in NCL.
- $ \max(x) $: The maximum value of the field across all GPs in NCL.
- $ y_j' $: The normalized value.

We want a higher score to be a good thing, so some metrics will need 'inverting'. For example, the percentage of people living with hypertension would need inverting (a lower percentage in this case indicates better performance). In this case, the formula is:
$$
y_j' = 1 - \frac{y_j - \min(y)}{\max(y) - \min(y)} = \frac{\max(y) - y_j}{\max(y) - \min(y)} 
$$

The dictionary `params.performance_metrics` contains an `'invert'` key for each metric specifying whether the metric should be inverted

In [ ]:
normalised_metrics = []
for i, (metric, metric_info) in enumerate(params.performance_metrics.items()):
    normalised_metrics.append(f'{metric}_norm')
    if metric_info['invert']:
        df_ncl_median_imputed[f'{metric}_norm'] = 1 - utils.min_max_normalisation(df_ncl_median_imputed[metric])
    else:
        df_ncl_median_imputed[f'{metric}_norm'] = utils.min_max_normalisation(df_ncl_median_imputed[metric])

Notice how the range for all the normalised metrics is $(0, 1)$.

In [ ]:
df_ncl_median_imputed[normalised_metrics].describe()

### Assign weights to each performance metric

This is where I would seek stakeholder engagement to understand which performance measures we are most interested in. These would then be assigned more weight.

For this analysis, I have given slightly more weight to efficiency metrics like 'Appointments per GP' and 'Attendance Rate'. 
Missed appointments are a significant cost burden to the NHS while the number of appointments provided by each GP is a good indicator of overall efficiency.

Please see the parameters.py file for the full set of weights used in this analysis.

### Obtain Composite Score

In [ ]:
df_performance_metrics = pd.DataFrame(params.performance_metrics).T

In [ ]:
df_ncl_median_imputed['performance_score'] = df_ncl_median_imputed[normalised_metrics].mul(list(df_performance_metrics.weight / df_performance_metrics.weight.sum()), axis = 1).sum(axis = 1)
df_ncl_median_imputed['performance_score'].describe()

In [ ]:
df_ncl_gps_ranked = df_ncl_median_imputed.sort_values(by = 'performance_score', ascending = True).reset_index(drop = True)

#### Lowest performing GPs in NCL

In [ ]:
df_low_performers = df_ncl_gps_ranked[0:4][['gp_code', 'performance_score', 'IMD2019', 'patients_per_gp'] + normalised_metrics]
df_low_performers.set_index('gp_code', inplace = True)

df_low_performers['gp_name'] = df_low_performers.apply(lambda row : utils.get_gp_name(row.index), axis = 1)
df_low_performers

### Analysis of Composite Score

#### Composite Score Distribution

In [ ]:
plt.figure(figsize = (20, 10))
plt.bar(df_ncl_gps_ranked.index, df_ncl_gps_ranked.performance_score)

In [ ]:
plt.hist(df_ncl_median_imputed.performance_score, bins = 15)
plt.xlim(0, 1)

plt.savefig(params.OUTPUTS_DIR / 'performance-score-distribution.png')

#### Radar Chart

Used to visualise the performance metrics for the lowest performing GPs.
These are compared to the maximum and average values of the performance metrics across NCL.  

In [ ]:
# Function to plot radar chart for a specific GP practice
def plot_radar_chart(df):
    # Metrics and values for the selected GP
    metrics = normalised_metrics
    metric_display_names = [params.column_display_names[metric] for metric, info in params.performance_metrics.items()]
    # Prepare for radar chart
    angles = np.linspace(0, 2 * np.pi, len(metrics), endpoint=False).tolist()
    angles += angles[:1]

    average_values = df_ncl_median_imputed[normalised_metrics].mean(axis=0).values # Average values for each metric in NCL
    high_values = df_ncl_median_imputed[normalised_metrics].max(axis=0).values  # Maximum values for each metric in NCL
    average_values = np.append(average_values, average_values[0])
    high_values = np.append(high_values, high_values[0])

    # Create the radar chart
    fig, ax = plt.subplots(math.ceil(len(df)/2), 2, figsize=(25, 25), subplot_kw=dict(polar=True))
    for i, gp_name in enumerate(df.index):
        values = df.loc[gp_name, normalised_metrics].values
        # Extend values to close the radar chart
        values = np.append(values, values[0])
        
        #ax[i].title(f"Performance Metrics for {gp_name}", fontsize=14)
        ax[i//2, i%2].fill(angles, high_values, color='green', alpha=0.2, label='Metric Max')
        ax[i//2, i%2].fill(angles, average_values, color='blue', alpha=0.2, label='Metric Average')
        ax[i//2, i%2].fill(angles, values, color='red', alpha=0.4, label=f'{gp_name}')
        ax[i//2, i%2].set_yticks([])  # Remove radial ticks
        ax[i//2, i%2].set_xticks(angles[:-1])  # Set metric labels
        ax[i//2, i%2].set_xticklabels(metric_display_names, fontsize=20)
        ax[i//2, i%2].legend(loc='upper left', bbox_to_anchor=(0.9, 1), fontsize=20)
    plt.tight_layout()
    plt.savefig(params.OUTPUTS_DIR / 'radar-chart.png', bbox_inches='tight')
    plt.show()
    

# Plot for a specific GP
plot_radar_chart(df_low_performers)

#### Boxplots of the Composite Score and Individual Metrics

In [ ]:
metrics_to_boxplot = ['performance_score'] + list(params.performance_metrics.keys())
metrics_to_boxplot.remove('overall_coded')
fig, ax = plt.subplots(math.ceil(len(metrics_to_boxplot)/3), 3, figsize=(10,15))

for i, metric in enumerate(metrics_to_boxplot):
    if metric == 'performance_score':
        df_boxplot = df_ncl_median_imputed[metric].rename('Composite Performance Score')
    else:
        df_boxplot = df_ncl_median_imputed[metric].rename(params.column_display_names[metric])
    sns.boxplot(df_boxplot, ax=ax[i//3, i%3], width = 0.2)

plt.tight_layout()

plt.savefig(params.OUTPUTS_DIR / 'performance-boxplot.png')

#### Map Plot

Plots GPs on a map.
The colour of the point on the map gives an indication of the performance rating (composite score) for the practice.

In [ ]:
### WARNING: This cell may take a long time to run (approx. 3 minutes)

# Geocode the postcodes
geolocator = Nominatim(user_agent="gp_performance_mapper")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

df_ncl_median_imputed['location'] = df_ncl_median_imputed['postcode'].apply(geocode)
df_ncl_median_imputed['point'] = df_ncl_median_imputed['location'].apply(lambda loc: tuple(loc.point) if loc else None)

# Drop rows with missing geocoded data
df_ncl_median_imputed = df_ncl_median_imputed.dropna(subset=['point'])

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame(df_ncl_median_imputed, geometry=gpd.points_from_xy(df_ncl_median_imputed['point'].apply(lambda x: x[1]), df_ncl_median_imputed['point'].apply(lambda x: x[0])))
gdf.crs = "EPSG:4326"

In [ ]:
# Plotting
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
gdf.plot(column='performance_score', ax=ax, legend=False, cmap='coolwarm', markersize=50)

# Add basemap
ctx.add_basemap(ax, crs=gdf.crs, source=ctx.providers.CartoDB.Positron)

# Add colorbar with label
sm = plt.cm.ScalarMappable(cmap='coolwarm', norm=plt.Normalize(vmin=gdf['performance_score'].min(), vmax=gdf['performance_score'].max()))
sm._A = []
cbar = plt.colorbar(sm, ax=ax)
cbar.set_label('Performance Score')

# Add title and labels
plt.title('GP Performance Map')
plt.xlabel('Longitude')
plt.ylabel('Latitude')

plt.savefig(params.OUTPUTS_DIR / 'performance-map.png')

# Show plot
plt.show()

#### Correlation Matrix

In [ ]:
cmap = sns.diverging_palette(230,20,as_cmap=True)

#plt.figure(figsize=(20, 10))
sns.heatmap(df_ncl[['IMD2019', 'patients_per_gp'] + performance_metrics].corr(), cmap=cmap)
#plt.tight_layout()

plt.savefig(params.OUTPUTS_DIR / 'correlation-heatmap.png', bbox_inches='tight')